In [1]:
import architecture
import data_setup
import engine
import os 
import torch

/Users/lam.nguyen/Desktop/GithubClone/TI_Final/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Set up folder and data zone

In [2]:
# # Set up folder and extract files in zip
# data_setup.extractzip()

In [3]:
# data_setup.concat_files("train")

In [4]:
# data_setup.concat_files("test")

# Testing zone

In [5]:
# # Read and change data into DataLoader
# train_dataloader = data_setup.create_dataloaders(os.path.join("..", "Data", "train", "FLC_data.csv"), 
#                                                  target_col='close', 
#                                                  batch_size=64)


In [6]:
# next(iter(train_dataloader))[0].shape

In [7]:
# # Initialise model
# model = architecture.SPP(input_size=12,
#                          hidden_size=64,
#                          output_shape=1)

In [8]:
# # Loss function and optimizer
# loss_func = torch.nn.MSELoss()
# optimizer = torch.optim.Adam(params=model.parameters(),
#                              lr=0.01)

In [9]:
# from torchinfo import summary

# summary(model,
#         input_size=(64, 12),
#         col_names = ['input_size', 'output_size', 'num_params', 'trainable'],
#         row_settings = ["var_names"])

In [10]:
# writer = engine.create_writer(experiment_name="SPP",
#                               model_name="base")

In [11]:
# # Training
# results = engine.train(model=model,
#                        train_dataloader=train_dataloader,
#                        loss_func=loss_func,
#                        epochs = 5,
#                        optimizer=optimizer)
#                     #    writer=writer)

# Training

In [12]:
# import pandas as pd
# data = pd.read_csv(os.path.join("..", "Data", "train", "StockData_train.csv"))
# samples = data.sample(n=50000, ignore_index=True).drop('Unnamed: 0', axis = 1)
# samples.to_csv(os.path.join("..", "Data", "train", "StockData50_train.csv"))

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [14]:
# Read and change data into DataLoader
train_dataloader = data_setup.create_dataloaders(train_path=os.path.join("..", "Data", "train", "StockData32_train.csv"), 
                                                 target_col='close', 
                                                 batch_size=128)


In [15]:
# Initialise model
model = architecture.SPP(input_size=12,
                         hidden_size=256,
                         output_shape=1).to(device)

In [16]:
# Loss function and optimizer
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=0.001)

In [17]:
from torchinfo import summary

summary(model, 
        input_size=(256, 12),
        col_names = ['input_size', 'output_size', 'num_params', 'trainable'],
        row_settings = ["var_names"])

Layer (type (var_name))                  Input Shape               Output Shape              Param #                   Trainable
SPP (SPP)                                [256, 12]                 [256, 1]                  --                        True
├─BNLSTM (bnlstm1)                       [256, 12]                 [256, 256]                512                       True
│    └─BNLSTMCell (bnlstmcell)           --                        [256, 256]                275,456                   True
│    │    └─BatchNorm1d (bn_hidden)      [256, 1024]               [256, 1024]               2,048                     True
│    │    └─BatchNorm1d (bn_input)       [256, 1024]               [256, 1024]               2,048                     True
│    │    └─BatchNorm1d (bn_c)           [256, 256]                [256, 256]                512                       True
├─Dropout (dropout1)                     [256, 256]                [256, 256]                --                        --
├─BNL

In [18]:
# writer = engine.create_writer(experiment_name="development",
#                               model_name="bnlstm_five")

[INFO] An instane of SummaryWriter is created, saving to: ../log/2025-03-17/development/bnlstm_five


In [19]:
# Training
results = engine.train(model=model,
                       train_dataloader=train_dataloader,
                       loss_func=loss_func,
                       epochs = 50,
                       optimizer=optimizer,
                       writer=writer,
                       device=device)

  0%|          | 0/50 [00:00<?, ?it/s]

  2%|▏         | 1/50 [01:12<58:53, 72.11s/it]

Epoch: 1 | Train loss: 16.5558 | Train evaluation score: 4.0689


  4%|▍         | 2/50 [02:22<56:40, 70.83s/it]

Epoch: 2 | Train loss: 4.4242 | Train evaluation score: 2.1034


...

Epoch: 48 | Train loss: 0.1358 | Train evaluation score: 0.3685


 98%|█████████▊| 49/50 [1:01:19<01:18, 78.32s/it]

Epoch: 49 | Train loss: 0.1311 | Train evaluation score: 0.3621


100%|██████████| 50/50 [1:02:32<00:00, 75.05s/it]

Epoch: 50 | Train loss: 0.1297 | Train evaluation score: 0.3602


In [20]:
engine.save_model(model=model,
                  model_name="bnlstmfive32_256_128_0001",
                  save_folder=os.path.join("..", "model"))

[INFO] Model is saved successfully to ../model/bnlstmfive32_256_128_0001


'../model/bnlstmfive32_256_128_0001'

In [21]:
import pandas as pd
tosave = pd.DataFrame(results).T
tosave.columns = ['Loss', 'Eval_score']
tosave.to_csv(os.path.join("..", "log_df", "bnlstmfive32_256_128_0001.csv"))